In [60]:
import pandas as pd
import numpy as np
import nltk, datetime, re
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz, process
from IPython.display import HTML

In [125]:
#Run notebook in conda env lexnlp
import lexnlp.extract.en.definitions
import lexnlp.extract.en.amounts
import lexnlp.extract.en.regulations

/Users/katherinewood/anaconda3/envs/lexnlp/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [25]:
def test_all_options(string1, string2):
    ratio = fuzz.ratio(string1, string2)
    partial_ratio = fuzz.partial_ratio(string1, string2)
    token_sort = fuzz.token_sort_ratio(string1, string2)
    token_set = fuzz.token_set_ratio(string1, string2)
    print(string1 + ', ' + string2 + '\n' +
        'ratio: ' + str(ratio) + '\n'
        'partial_ratio: ' + str(ratio) + '\n'
        'token_sort: ' + str(ratio) + '\n'
        'token_set: ' + str(ratio) + '\n')

In [54]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('expand_frame_repr', False)

In [146]:
recalls = pd.read_csv('recalls.csv', 
                      dtype={'RecallID':'Int64', 'RecallNumber': str, 'RecallDate': str,
                             'Description': str, 'URL': str, 'Title': str, 'ConsumerContact': str,
                             'LastPublishDate': str, 'Images': 'object', 'SoldAtLabel': str,
                             'Distributors_CompanyID': 'Int64', 'DistributorsSName': str, 
                             'Hazards_HazardType': str, 'Hazards_HazardTypeID': str, 'Hazards_Name': str,
                             'Importers_CompanyID': 'Int64', 'Importers_Name': str, 'Inconjunctions_URL': str,
                             'Injuries_Name': str, 'ManufacturerCountries_Country': str, 
                             'Manufacturers_CompanyID': 'Int64', 'Manufacturers_Name': str, 'ProductUPCs_UPC': str,
                             'Products_CategoryID': 'Int64', 'Products_Description': str, 'Products_Model': str,
                             'Products_Name': str, 'Products_NumberOfUnits': str, 'Products_Type': str,
                             'Remedies_Name': str, 'RemedyOptions_Option': str, 'Retailers_CompanyID': str,
                             'Retailers_Name': str})

In [150]:
#Munge and infill whatever information we can extract from the recall descriptions

##UPCs loaded as strings; clean for whitespace & non-numeric chars
recalls['ProductUPCs_UPC'] = recalls['ProductUPCs_UPC'].str.replace(' |-|\.', '')
recalls = recalls.rename(columns={'ProductUPCs_UPC': 'UPC'})
#Extract unit numbers from string phrases (e.g. "About 35")
num_units = recalls['Products_NumberOfUnits'].str.replace(',', '')
num_units = num_units.str.extract(r'(\d+)', expand=False).astype('float')
recalls['Products_NumberOfUnits'] = num_units
#Extract total number of complaints from the string column
#Parse dates from strings
recalls['RecallDate'] = pd.to_datetime(recalls['RecallDate'])
recalls['LastPublishDate'] = pd.to_datetime(recalls['LastPublishDate'])
#Break the standardized titles into helpful fields; standardized
#titles take the form "[Company] recalls [product] due to [hazard]"
titles = recalls['Title'].str.split('[Announce]?[s]?Recall[s]?[ed]?|Due to', expand=True)
titles = titles.rename(columns={0: 'CompanyShortname', 1: 'ProductsShortname', 2: 'HazardAlt'})
recalls = pd.concat([recalls, titles], axis=1)

In [68]:
#Vectorize over a series of strings
def extract_probable_specifiers(text):
    if pd.isnull(text):
        text=''
    pattern = r"(([0-9A-Z])+[a-z]*([\\-]?[\\.*]?[0-9A-Z]*)*){2,}"
    matches = re.finditer(pattern, text)
    unique_matches = set([match.group() for matchNum, match in enumerate(matches)])
    return list(unique_matches)

In [182]:
# TODO combine vornado and bosch for a test set
reports = pd.concat([v_reports, b_reports], axis=0, sort=False)
recalls = pd.concat([v_recalls, b_recalls], axis=0, sort=False)

#Funnel match
#Preparation: extract possible brands from the fields likely to contain them, for both the reports and the recalls.
#Brands may not be referred to by a consistent name across the two datasets.
brand_from_comments = [report[1] if not isinstance(report, float) else '' for report in reports['Company Comments'].str.split('\\) |:')]
reports['candidate_brand'] = list(zip(reports['Brand'], 
                                           reports['Manufacturer / Importer / Private Labeler Name'],
                                           brand_from_comments))
recalls['candidate_brand'] = list(zip(recalls['CompanyShortname'], recalls['Manufacturers_Name'],
                                     recalls['Distributors_Name'], recalls['Importers_Name']))

#Preparation: extract alphanumeric strings that are likely to be model numbers, serial numbers, or UPCs.
#Recall notices are very unlikely to have the dedicated fields populated, but tend to mention them in the
#text of the recall announcement.
#Reports tend to have this information in the dedicated fields, but as a precaution we also try to pull it
#from the unstructured text.

reports['specifiers'] = [extract_probable_specifiers(report) for report in 
                         [reports['Product Description'] + ' '+ reports['Incident Description']][0]]

recalls['specifiers'] = [extract_probable_specifiers(recall) for recall in recalls['Description']]

#Phase 1
#For each complaint:
#for each candidate brand: 'CompanyShortname', 'Manufacturers_Name', 'Distributors_Name', 'Importers_Name'
#fuzzy match to the possible brands list from each recall notice
#if any matches score > 50, save recall as candidate (add a column that contains a list of probable recall IDs)
#if no matches score > 50, label complaint as "no recall"
possible_matches = []
for i in range(len(reports)):
    report = reports.iloc[i, :]
    match_ids = []
    for c in report['candidate_brand']:
        if pd.isnull(c) or not c:
            continue
        else:
            match_ids += [recalls.iloc[r]['RecallID'] for r in range(len(recalls)) if
                        (pd.Series(list(zip(*process.extract(c, recalls.iloc[r]['candidate_brand'])))[1]) > 60).any()]
    possible_matches.append(list(set(match_ids)) if match_ids else 0)

reports['brand_match_candidates'] = possible_matches

#Phase 2
#For all complaints that have candidate recall numbers:
#For each candidate recall:
#fuzzy match to possible products using product type from report & product name from recall; retain match if score > 50
#on any
#finally, take the specifiers extracted from the recall description and look for an exact match in the following order:
#Model name or number
#serial number
#UPC
#specifiers extracted from the product description
#speficiers extracted from incident description

In [183]:
reports

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,Manufacturer / Importer / Private Labeler Name,Brand,Model Name or Number,Serial Number,UPC,Date Manufactured,Manufacturer Date Code,Retailer,Retailer State,Purchase Date,Purchase Date Is Estimate,Incident Description,City,State,ZIP,Location,(Primary) Victim Severity,(Primary) Victim's Gender,My Relation To The (Primary) Victim,(Primary) Victim's Age (years),Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers,candidate_brand,specifiers,brand_match_candidates
1732,20190518-5D65D-2147380898,5/18/2019,6/7/2019,6/21/2019,Consumer,Steamfast SF-275 Heavy-Duty Canister Steam Cle...,Home Maintenance and Structures,Other,"Appliances, Other and Not Specified (482)",482,Vornado Air LLC,Steamfast SF-275,SF-275,NaN,NaN,NaN,NaN,Costco Wholesale,NaN,2/5/2019,NaN,After steam cleaning shower for approximately ...,San Diego,California,92117,Home/Apartment/Condominium,"Injury, No First Aid or Medical Attention Rece...",Female,Self,43,Yes,No,NaN,NaN,No,No,Yes,I still have the product. I plan to contact th...,(6/12/2019) Vornado Air LLC: Thank you for the...,NaN,"(Steamfast SF-275, Vornado Air LLC, Vornado Ai...","[19, 17th., 12, 30-45, SF-275, Heavy-Duty, 201...","[2662, 8298, 8240, 1748, 8381]"
2238,20190316-1D44C-2147382171,3/16/2019,3/29/2019,4/12/2019,Consumer,"Vornado AVH10 Vortex Heater, a portable space ...",Home Maintenance and Structures,"Heating, Ventilation & Air Conditioning",Portable Electric Heaters (348),348,Vornado Air LLC,Vornado,AVH10,Will provide later - box is in storage,'Will provide later,NaN,Unknown,Amazon.com,NaN,1/31/2019,NaN,"On January 31, 2019, I purchased a Vornado AVH...",Carlsbad,California,92011,Other Public Property/Office,"Incident, No Injury",Unspecified,Self,NaN,Yes,No,NaN,NaN,No,Yes,NaN,"I contacted Vornado on or about March 6, 2019 ...",(3/29/2019) Vornado Air LLC: Thank you for the...,NaN,"(Vornado, Vornado Air LLC, Vornado Air LLC)","[65, 72, 2019, 31, AVH10]","[2662, 8298, 8240, 1748, 8381]"
2777,20190126-E7DAD-2147383091,1/27/2019,2/12/2019,2/28/2019,Public Safety Entity,Small portable electric space heater reportedl...,Home Maintenance and Structures,"Heating, Ventilation & Air Conditioning",Portable Electric Heaters (348),348,Vornado Air LLC,Vornado,Vortex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Structural fire reported at 0204 hrs on 01/26/...,Hauppauge,New York,11788,Home/Apartment/Condominium,"Injury, Hospital Admission",Male,Unspecified,62,No,No,NaN,NaN,No,No,No,NaN,(2/26/2019) Vornado Air LLC: Thank you for the...,NaN,"(Vornado , Vornado Air LLC, Vornado Air LLC)","[2019., 01, 26, 0204]","[2662, 8298, 8240, 1748, 8381]"
3084,20181210-542B3-2147383846,12/10/2018,12/17/2018,2/4/2019,Consumer,"Small Space heater. (10"")",Home Maintenance and Structures,"Heating, Ventilation & Air Conditioning",Portable Electric Heaters (348),348,Vornado Air LLC,Vornado,Model pvh,12489,NaN,12/11/2011,8/11,bed bath and beyond,California,12/11/2011,Yes,On/Off switch does not turn unit off. Light go...,Los Angeles,California,90068,Home/Apartment/Condominium,"Incident, No Injury",Unknown,Unspecified,NaN,Yes,No,NaN,NaN,No,Yes,NaN,They said that this product has not been re-ca...,(12/17/2018) Vornado Air LLC: Thank you for th...,NaN,"(Vornado, Vornado Air LLC, Vornado Air LLC)",[10],"[2662, 8298, 8240, 1748, 8381]"
3323,20181102-A17FD-2147384672,11/2/2018,11/9/2018,11/27/2018,Consumer,Space heater,Home Maintenance and Structures,"Heating, Ventilation & Air Conditioning",Portable Electric Heaters (348),348,Vornado Air LLC,VORNADO,VORNADO TYPE AVH2 DIGITAL 120V,OCT17 - 22173,NaN,NaN,NaN,Costco,NaN,11/1/2017,Yes,Vornado space heater type AVH2 DIGIT

In [172]:
possible_matches

[[2821,
  518,
  138,
  2834,
  1179,
  6057,
  8240,
  2355,
  440,
  6075,
  5053,
  8381,
  1087,
  6725,
  1748,
  8027,
  6492,
  3292,
  3293,
  2662,
  8298,
  4330,
  5867,
  6510,
  3697,
  2933,
  1791],
 [2821,
  518,
  138,
  2834,
  1179,
  6057,
  8240,
  2355,
  440,
  6075,
  5053,
  8381,
  1087,
  6725,
  1748,
  8027,
  6492,
  3292,
  3293,
  2662,
  8298,
  4330,
  5867,
  6510,
  3697,
  2933,
  1791],
 [2821,
  518,
  138,
  2834,
  1179,
  6057,
  8240,
  2355,
  440,
  6075,
  5053,
  8381,
  1087,
  6725,
  1748,
  8027,
  6492,
  3292,
  3293,
  2662,
  8298,
  4330,
  5867,
  6510,
  3697,
  2933,
  1791],
 [2821,
  518,
  138,
  2834,
  1179,
  6057,
  8240,
  2355,
  440,
  6075,
  5053,
  8381,
  1087,
  6725,
  1748,
  8027,
  6492,
  3292,
  3293,
  2662,
  8298,
  4330,
  5867,
  6510,
  3697,
  2933,
  1791],
 [2821,
  518,
  138,
  2834,
  1179,
  6057,
  8240,
  2355,
  440,
  6075,
  5053,
  8381,
  1087,
  6725,
  1748,
  8027,
  6492,
  3292,
  3

In [469]:
recalls['CompanyShortname'].iloc[0:10]

0            The Thompson’s Company 
1    Boston Warehouse Trading Corp. 
2                      Libbey Glass 
3        Thesaurus Global Marketing 
4            Bosch Thermotechnology 
5                          Hallmark 
6                         BCI Burke 
7                               BMC 
8                          Toysmith 
9               Hillsdale Furniture 
Name: CompanyShortname, dtype: object

In [147]:
reports = pd.read_csv('SPDB/IncidentReports.csv', encoding="ISO-8859-1", dtype='object')

In [142]:
def get_matched_brands(brand, reports, recalls):
    brand_reports = reports[reports['Manufacturer / Importer / Private Labeler Name'].str.contains(brand, case=False) 
                            | reports['Brand'].str.contains(brand, case=False)
                            | reports['Incident Description'].str.contains(brand, case=False)]
    brand_recalls = recalls[recalls['CompanyShortname'].str.contains(brand, case=False) 
                            | recalls['Manufacturers_Name'].str.contains(brand, case=False) 
                            | recalls['Importers_Name'].str.contains(brand, case=False) 
                            | recalls['Distributors_Name'].str.contains(brand, case=False) 
                            | recalls['Retailers_Name'].str.contains(brand, case=False)]
    return (brand_reports, brand_recalls)


In [151]:
v_reports, v_recalls = get_matched_brands('Vornado', reports, recalls)

In [152]:
b_reports, b_recalls = get_matched_brands('Bosch', reports, recalls)

In [153]:
b_reports

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,Manufacturer / Importer / Private Labeler Name,Brand,Model Name or Number,Serial Number,UPC,Date Manufactured,Manufacturer Date Code,Retailer,Retailer State,Purchase Date,Purchase Date Is Estimate,Incident Description,City,State,ZIP,Location,(Primary) Victim Severity,(Primary) Victim's Gender,My Relation To The (Primary) Victim,(Primary) Victim's Age (years),Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
10,20191202-0EE2E-2147376236,12/2/2019,12/27/2019,1/10/2020,Consumer,Bosch Dishwasher Model # SHEM63W55N/10,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHEM63W55N/10,FD980402890,NaN,NaN,NaN,Best Buy,North Carolina,4/28/2018,Yes,Bosch Dishwasher power cord caught on fire. Th...,Raleigh,North Carolina,27608,Home/Apartment/Condominium,"Incident, No Injury",Unknown,Unspecified,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,I contacted the manufacturer and the customer ...,(1/9/2020) BSH HOME APPLIANCES CORPORATION: Th...,NaN
90,20191111-EC57F-2147376634,11/11/2019,12/16/2019,1/1/2020,Consumer,BOSCH Dishwasher,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,BOSCH,SHU43C05UC/17,FD840500303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dishwasher was running a normal cycle when a h...,Carnation,Washington,98014,Home/Apartment/Condominium,"Incident, No Injury",Male,Self,29,Yes,No,NaN,NaN,No,No,NaN,NaN,(1/9/2020) BSH HOME APPLIANCES CORPORATION: Th...,NaN
113,20191113-6DFF6-2147376590,11/13/2019,12/13/2019,12/31/2019,Consumer,Bosch dishwasher with controls on the top. 3 ...,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHP65TL5UC/09,FD961101714,NaN,NaN,FD961101714,Alber Lee Appliances,Washington,3/8/2017,Yes,Wife went to use the Bosh Dishwasher on the af...,Seattle,Washington,98136,Home/Apartment/Condominium,"Incident, No Injury",Female,My Spouse,NaN,Yes,No,NaN,NaN,No,Yes,NaN,Called about getting it serviced. Found out a...,(1/9/2020) BSH HOME APPLIANCES CORPORATION: Th...,NaN
131,20191111-4EB92-2147376627,11/11/2019,12/13/2019,12/31/2019,Consumer,Bosch 800 series electric wall oven,Kitchen,Appliances,Electric Ranges or Ovens (Excl Counter-top Ove...,278,BSH HOME APPLIANCES CORPORATION,Bosch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Danger when opening 2019 Bosch 800 series sing...,Brooklyn,New York,11217,Home/Apartment/Condominium,"Injury, No First Aid or Medical Attention Rece...",Male,Self,48,Yes,No,NaN,NaN,No,No,No,NaN,(1/9/2020) BSH HOME APPLIANCES CORPORATION: BS...,NaN
267,20191101-00021-2147376817,11/1/2019,11/19/2019,12/4/2019,Consumer,"Bosch Integra series dishwasher 18"" width, sta...",Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SPX5ES55UC/07,FD9205,'00039,NaN,NaN,Appliances Connection,NaN,12/20/2012,Yes,Dishwasher Power cord caught fire,BOSTON,Massachusetts,02118,Home/Apartment/Condominium,"Incident, No Injury",Unspecified,Self,57,Yes,No,NaN,NaN,No,No,Yes,It was destroyed by firemen.,(12/5/2019) BSH HOME APPLIANCES CORPORATION: T...,NaN
328,20191015-16DFD-2147377150,10/15/2019,11/12/2019,11/26/2019,Consumer,Bosch Dishwasher Model #SHX68T55UC/02\r\n\r\nO...,Kitchen,Appliances,Dishwashers (214),214,BSH HOME APPLIANCES CORPORATION,Bosch,SHX68T55UC/02,FD9601,NaN,NaN,03626,PC Richard and Son,New Jersey,1/17/2016,NaN,Bosch Dishwasher Model #SHX68T55UC/02\r\n\r\nO...,Stewartsville,New Jersey,08886,Home/Apartment/Condominium,"Incident, No Injury",Unspecified,Self,NaN,Yes,No,NaN,NaN,No,Yes,NaN,The manufacturer is saying our serial number i...,(12/5/2019) BSH HOME APPLIANCES CORPORATION: A...,NaN
444,20191006-F6864-2147377298,10/6/2019,10/25/2019,11/8/20

In [154]:
recalls

,RecallID,RecallNumber,RecallDate,Description,URL,Title,ConsumerContact,LastPublishDate,Images,SoldAtLabel,Distributors_CompanyID,Distributors_Name,Hazards_HazardType,Hazards_HazardTypeID,Hazards_Name,Importers_CompanyID,Importers_Name,Inconjunctions_URL,Injuries_Name,ManufacturerCountries_Country,Manufacturers_CompanyID,Manufacturers_Name,UPC,Products_CategoryID,Products_Description,Products_Model,Products_Name,Products_NumberOfUnits,Products_Type,Remedies_Name,RemedyOptions_Option,Retailers_CompanyID,Retailers_Name,CompanyShortname,ProductsShortname,HazardAlt,3
0,8726,20056,2020-01-14,This recall involves Thompson’s WaterSeal wate...,https://www.cpsc.gov/Recalls/2020/The-Thompson...,The Thompson’s Company Recalls Aerosol Waterpr...,The Thompson’s Company toll-free at 888-304-37...,2020-01-14,[{'URL': 'https://www.cpsc.gov/s3fs-public/Cap...,NaN,NaN,NaN,NaN,NaN,The contents of the cans can react with the pa...,NaN,NaN,NaN,The Thompson’s Company received approximately ...,United States,NaN,"The Thompson’s Company, of Cleveland, Ohio",NaN,NaN,NaN,NaN,Thompson’s® WaterSeal® Waterproofing Wood Prot...,852000.0,NaN,Consumers should immediately stop using Thomps...,Refund,NaN,"Lowe’s Home Improvement, Walmart, Menards, Hom...",The Thompson’s Company,Aerosol Waterproofing Wood and Masonry Protec...,Fire Hazard,None
1,8723,20050,2020-01-09,This recall involves Holiday Travel mugs with ...,https://www.cpsc.gov/Recalls/2020/Boston-Wareh...,Boston Warehouse Trading Corp. Recalls Holiday...,Boston Warehouse Trading Corp. toll-free at 88...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,The mugs are mislabeled as microwave safe. If ...,NaN,"Meijer Distribution Inc., of Grand Rapids, Mich.",NaN,The firm has received one report of sparks whe...,China,NaN,"Boston Warehouse Trading Corp., of Norwood, Mass.",NaN,NaN,NaN,NaN,Holiday Travel Mugs,2400.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,Exclusively at Meijer stores nationwide from O...,Boston Warehouse Trading Corp.,Holiday Travel Mugs,Fire Hazard; Sold Exclusively at Meijer Stores,None
2,8724,20051,2020-01-09,This recall involves the Libbey Glass 33.5 oz....,https://www.cpsc.gov/Recalls/2020/Libbey-Glass...,Libbey Glass Recalls Milk Bottles Due to Lacer...,Libbey Glass at 800-982-7063 between 8 a.m. an...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,"The bottles can break unexpectedly during use,...",NaN,"Libbey Glass Inc., of Toledo, Ohio",NaN,None Reported,China,NaN,NaN,NaN,NaN,NaN,NaN,33.5 oz. Milk Bottles,44300.0,NaN,Foodservice establishments and customers shoul...,Refund,NaN,Libby sold the recalled bottles to various foo...,Libbey Glass,Milk Bottles,Laceration Hazard,None
3,8725,20712,2020-01-09,This recall involves Little Bambino 4 in 1 can...,https://www.cpsc.gov/Recalls/2020/Thesaurus-Gl...,Thesaurus Global Marketing Recalls Tricycles D...,Little Bambino toll-free at 866-633-8202 from ...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,Paint on the canopy’s frame contains levels of...,NaN,"Thesaurus Global Marketing Inc., of Doral, FL",NaN,None reported,China,NaN,NaN,NaN,NaN,NaN,NaN,Little Bambino 4 in 1 canopy children’s tricycles,370.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,Amazon.com from October 2018 through June 2019...,Thesaurus Global Marketing,Tricycles,Violation of the Federal Lead Paint Ban; Risk...,Alert)
4,8721,20046,2019-12-20,This recall involves Buderus brand GB125-35 oi...,https://www.cpsc.gov/Recalls/2020/Bosch-Thermo...,Bosch Thermotechnology Recalls Buderus Boilers...,Bosch Thermotechnology at 800-323-1943 from 8 ...,2019-12-20,[{'URL': 'https://www.cpsc.gov/s3fs-public/Scr...,NaN,NaN,"Bosch Thermotechnology Corp., of Watertown, Mass.",NaN,NaN,"The siphon can become blocked, leading to a de...",NaN,"Bosch Thermotechnology Corp., of Watertown, Mass.",NaN,None reported in the U.S.,Germany,NaN,"Bosch Thermotechnik